In [2]:
pip install torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
train_data = datasets.MNIST(
    root = 'data',
    train = True ,
    transform = ToTensor(),
    download = True
    )


test_data = datasets.MNIST(
    root = 'data',
    train = False ,
    transform = ToTensor(),
    download = True
    )

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 463kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.40MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.88MB/s]


In [7]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(train_data, batch_size=100, shuffle=True, num_workers=1),
    'test': DataLoader(test_data, batch_size=100, shuffle=True, num_workers=1),
}

In [8]:
loaders


{'train': <torch.utils.data.dataloader.DataLoader at 0x7a249ffd5e50>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7a24a01a3c10>}

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d() #normalization, deactivates layer
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10) # 10 because 10 digits and softmax function will be used - prob for every digit

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)

    return F.softmax(x)


In [13]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(loaders['train']):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 50 == 0 :
      print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')


def test():
  model.eval()

  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in loaders['test']:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_fn(output, target).item()
      pred = output.argmax(dim=1, keepdim = True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(loaders['test'].dataset)
  print(f'\nTest set: Average loss: {test_loss: .4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct/len(loaders["test"].dataset):.0f}%\n)')


In [14]:
for epoch in range(1, 11):
  train(epoch)
  test()

<ipython-input-11-9f545a38dd56>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.304203
Train Epoch: 1 [5000/60000 (8%)]	2.086371
Train Epoch: 1 [10000/60000 (17%)]	1.983278
Train Epoch: 1 [15000/60000 (25%)]	1.788386
Train Epoch: 1 [20000/60000 (33%)]	1.798816
Train Epoch: 1 [25000/60000 (42%)]	1.632899
Train Epoch: 1 [30000/60000 (50%)]	1.653893
Train Epoch: 1 [35000/60000 (58%)]	1.679186
Train Epoch: 1 [40000/60000 (67%)]	1.660423
Train Epoch: 1 [45000/60000 (75%)]	1.583556
Train Epoch: 1 [50000/60000 (83%)]	1.580196
Train Epoch: 1 [55000/60000 (92%)]	1.594623

Test set: Average loss:  0.0154, Accuracy 9193/10000 (92%
)
Train Epoch: 2 [0/60000 (0%)]	1.664144
Train Epoch: 2 [5000/60000 (8%)]	1.613623
Train Epoch: 2 [10000/60000 (17%)]	1.643492
Train Epoch: 2 [15000/60000 (25%)]	1.620567
Train Epoch: 2 [20000/60000 (33%)]	1.553500
Train Epoch: 2 [25000/60000 (42%)]	1.606344
Train Epoch: 2 [30000/60000 (50%)]	1.598264
Train Epoch: 2 [35000/60000 (58%)]	1.657364
Train Epoch: 2 [40000/60000 (67%)]	1.650057
Train Epoch: 2 [45000/60000 (